In [ ]:
import numpy as np
import pandas as pd

In [ ]:
movies = pd.read_csv('data/tmdb_5000_movies.csv')
credits = pd.read_csv('data/tmdb_5000_credits.csv')

In [ ]:
movies.head(1)

In [ ]:
credits.head(1)

In [ ]:
movies.describe()

In [ ]:
movies.merge(credits,on='title')

In [ ]:
movies.merge(credits,on='title').shape

In [ ]:
movies.shape

In [ ]:
credits.shape

In [ ]:
movies = movies.merge(credits,on='title')

In [ ]:
movies.head(1)

In [ ]:
movies.info()

In [ ]:
''' genres,movie_id,keywords,overview,title,cast,crew'''
movies = movies[['movie_id','title','overview','genres','keywords','cast','crew']]

In [ ]:
movies.head(1)

In [ ]:
movies['cast'].values[4]

In [ ]:
movies.isnull().sum()

In [ ]:
movies.dropna(inplace = True)

In [ ]:
movies.duplicated().sum()

In [ ]:
movies.iloc[0].genres

In [ ]:
import ast
ast.literal_eval('[{"id": 28, "name": "Action"}, {"id": 12, "name": "Adventure"}, {"id": 14, "name": "Fantasy"}, {"id": 878, "name": "Science Fiction"}]'
)

In [ ]:
def convert(obj):
    L = []
    for i in ast.literal_eval(obj):
        L.append(i['name'])
    return L

In [ ]:
movies['genres'] = movies['genres'].apply(convert)

In [ ]:
movies.head()

In [ ]:
movies['keywords'].apply(convert)

In [ ]:
movies['keywords'] = movies['keywords'].apply(convert)

In [ ]:
def convert3(obj):
    L = []
    counter = 0
    for i in ast.literal_eval(obj):
        if counter != 3:
            L.append(i['name'])
            counter+= 1
        else:
            break
    return L

In [ ]:
movies['cast'].apply(convert3)

In [ ]:
movies['cast'] = movies['cast'].apply(convert3)

In [ ]:
movies.head()

In [ ]:
def fetch_director(obj):
    L = []
    for i in ast.literal_eval(obj):
        if i['job'] == 'Director':
          L.append(i['name'])
    return L

In [ ]:
movies['crew'].apply(fetch_director)

In [ ]:
movies['crew'] = movies['crew'].apply(fetch_director)

In [ ]:
movies.head()

In [ ]:
movies['overview'][0]

In [ ]:
movies['overview'].apply(lambda x:x.split())

In [ ]:
movies['overview'] = movies['overview'].apply(lambda x:x.split())

In [ ]:
movies.head()

In [ ]:
movies['genres'].apply(lambda x:[i.replace(" ","") for i in x])

In [ ]:
movies['genres'] = movies['genres'].apply(lambda x:[i.replace(" ","") for i in x])
movies['keywords'] = movies['keywords'].apply(lambda x:[i.replace(" ","") for i in x])
movies['cast'] = movies['cast'].apply(lambda x:[i.replace(" ","") for i in x])
movies['crew'] = movies['crew'].apply(lambda x:[i.replace(" ","") for i in x])

In [ ]:
movies.head()

In [ ]:
movies['tags'] = movies['overview'] + movies['genres'] + movies['keywords'] + movies['cast'] + movies['crew']

In [ ]:
movies.head()

In [ ]:
new_df = movies[['movie_id','title','tags']]

In [ ]:
new_df.head()

In [ ]:
new_df['tags'].apply(lambda x:" ".join(x))

In [ ]:
new_df['tags'] = new_df['tags'].apply(lambda x:" ".join(x))

In [ ]:
new_df.head()

In [ ]:
new_df['tags'][0]

In [ ]:
new_df['tags'] = new_df['tags'].apply(lambda x:x.lower())

In [ ]:
new_df.head()

In [ ]:
new_df['tags'][0]

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=5000,stop_words='english')

In [ ]:
cv.fit_transform(new_df['tags']).toarray()

In [ ]:
vectors = cv.fit_transform(new_df['tags']).toarray()

In [ ]:
cv.get_feature_names_out()

In [ ]:
!pip install nltk
import nltk

In [ ]:
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()

In [ ]:
def stem(text):
    y=[]
    
    for i in text.split():
        y.append(ps.stem(i))
        
    return" ".join(y)

In [ ]:
stem('in the 22nd century, a paraplegic marine is dispatched to the moon pandora on a unique mission, but becomes torn between following orders and protecting an alien civilization. action adventure fantasy sciencefiction cultureclash future spacewar spacecolony society spacetravel futuristic romance space alien tribe alienplanet cgi marine soldier battle loveaffair antiwar powerrelations mindandsoul 3d samworthington zoesaldana sigourneyweaver jamescameron')

In [ ]:
new_df['tags'] = new_df['tags'].apply(stem)

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
similarity = cosine_similarity(vectors)

In [ ]:
similarity

In [ ]:
def recommend(movie):
    movie_index = new_df[new_df['title'] == movie].index[0]
    distances = similarity[movie_index]
    movies_list = sorted(list(enumerate(distances)), reverse=True, key=lambda x: x[1])[1:6]

    for i in movies_list:
        print(new_df.iloc[i[0]].title)

In [ ]:
recommend('Newlyweds')

In [ ]:
new_df.tail()

In [ ]:
import pickle

In [ ]:
pickle.dump(new_df,open('models/movie_list.pkl','wb'))
pickle.dump(similarity,open('models/similarity.pkl','wb'))


In [ ]:
pickle.dump(new_df.to_dict(),open('models/movie_dict.pkl','wb'))

In [ ]:
import pandas as pd
import requests
import pickle
from tqdm import tqdm

API_KEY = "8265bd1679663a7ea12ac168da84d2e8"
BASE_URL = "https://api.themoviedb.org/3/movie/{}?api_key={}"

movies = pd.read_csv("data/tmdb_5000_movies.csv")  # your dataset
poster_links = {}

for index, row in tqdm(movies.iterrows(), total=movies.shape[0]):
    movie_id = row["id"]
    url = BASE_URL.format(movie_id, API_KEY)

    try:
        response = requests.get(url, timeout=4)
        data = response.json()

        poster_path = data.get("poster_path")
        if poster_path:
            full_url = "https://image.tmdb.org/t/p/w500" + poster_path
        else:
            full_url = None

        poster_links[movie_id] = full_url

    except:
        poster_links[movie_id] = None

# Save poster dictionary
with open("models/posters.pkl", "wb") as f:
    pickle.dump(poster_links, f)

print("Posters saved: models/posters.pkl")
